In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('SentimentAnalysisDataset.csv', encoding='latin-1')

In [10]:
df=df.drop(columns=['textID','Country','Population -2020','Land Area (Km²)','Density (P/Km²)'])

In [11]:
df=df.dropna()

In [12]:
df

,text,sentiment,Time of Tweet,Age of User
0,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20
1,Shanghai is also really exciting (precisely -...,positive,noon,21-30
2,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45
3,happy bday!,positive,morning,46-60
4,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70
...,...,...,...,...
3529,"its at 3 am, im very tired but i can`t sleep ...",negative,noon,21-30
3530,All alone in this old house again. Thanks for...,positive,night,31-45
3531,I know what you mean. My little dog is sinkin...,negative,morning,46-60
3532,_sutra what is your next youtube video gonna b...,positive,noon,60-70


In [13]:
help(Conv1D)

Help on class Conv1D in module keras.layers.convolutional.conv1d:

class Conv1D(keras.layers.convolutional.base_conv.Conv)
 |  Conv1D(filters, kernel_size, strides=1, padding='valid', data_format='channels_last', dilation_rate=1, groups=1, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, **kwargs)
 |  
 |  1D convolution layer (e.g. temporal convolution).
 |  
 |  This layer creates a convolution kernel that is convolved
 |  with the layer input over a single spatial (or temporal) dimension
 |  to produce a tensor of outputs.
 |  If `use_bias` is True, a bias vector is created and added to the outputs.
 |  Finally, if `activation` is not `None`,
 |  it is applied to the outputs as well.
 |  
 |  When using this layer as the first layer in a model,
 |  provide an `input_shape` argument
 |  (tuple of integers or `None`, e.g

In [15]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

max_features = 10000
maxlen = 100

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['text'])
x_text = tokenizer.texts_to_sequences(df['text'])
x_text = pad_sequences(x_text, maxlen=maxlen)

categorical_features = ['Time of Tweet']

label_mapping = {'positive': 2, 'negative': 1, 'neutral': 0}
y = np.array([label_mapping[label] for label in df['sentiment']])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ])

x_features = preprocessor.fit_transform(df[categorical_features])

x_text_train, x_text_test, x_features_train, x_features_test, y_train, y_test = train_test_split(
    x_text, x_features, y, test_size=0.2, random_state=42
)

text_input = Input(shape=(maxlen,))
embedding_layer = Embedding(input_dim=max_features, output_dim=50, input_length=maxlen)(text_input)
conv1d_layer = Conv1D(128, 5, activation='relu')(embedding_layer)
maxpooling_layer = GlobalMaxPooling1D()(conv1d_layer)

numerical_input = Input(shape=(x_features.shape[1],))
concatenated_layer = Concatenate()([maxpooling_layer, numerical_input])
dense_layer = Dense(64, activation='sigmoid')(concatenated_layer)
output_layer = Dense(3, activation='softmax')(dense_layer)

model = Model(inputs=[text_input, numerical_input], outputs=output_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 64
epochs = 25
model.fit([x_text_train, x_features_train], y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

loss, accuracy = model.evaluate([x_text_test, x_features_test], y_test)
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")


Epoch 1/25
36/36 [==============================] - 1s 14ms/step - loss: 1.0858 - accuracy: 0.4060 - val_loss: 1.0984 - val_accuracy: 0.3869
Epoch 2/25
36/36 [==============================] - 0s 11ms/step - loss: 1.0633 - accuracy: 0.4237 - val_loss: 1.0708 - val_accuracy: 0.4329
Epoch 3/25
36/36 [==============================] - 0s 11ms/step - loss: 0.9354 - accuracy: 0.6218 - val_loss: 0.9391 - val_accuracy: 0.5177
Epoch 4/25
36/36 [==============================] - 0s 11ms/step - loss: 0.6378 - accuracy: 0.7824 - val_loss: 0.8599 - val_accuracy: 0.6078
Epoch 5/25
36/36 [==============================] - 0s 11ms/step - loss: 0.3534 - accuracy: 0.8978 - val_loss: 0.8975 - val_accuracy: 0.6237
Epoch 6/25
36/36 [==============================] - 0s 11ms/step - loss: 0.1739 - accuracy: 0.9642 - val_loss: 0.9757 - val_accuracy: 0.6131
Epoch 7/25
36/36 [==============================] - 0s 11ms/step - loss: 0.0851 - accuracy: 0.9912 - val_loss: 1.0494 - val_accuracy: 0.6095
Epoch 8/25
36